In [ ]:
def createBoard():
    initBoard = []
    for i in range(0,6):
        column = []
        for j in range(0,6):
            column.append("-")
        initBoard.append(column)
    return initBoard


def printBoard(board):
    for i in range(len(board[0])-1,-1,-1):
        printString = "|"
        for j in range(0,len(board)):
            printString = printString + " " + (board[j][i])
        printString = printString + " " + "|"
        print(printString)

def gameHasEnded(board):
    # Check rows
    for row in board:
        for i in range(3):
            if row[i] == row[i + 1] == row[i + 2] == row[i + 3] != '-':
                return row[i]  # Return the winner ('X' or 'O')

    # Check columns
    for col in range(6):
        for i in range(3):
            if board[i][col] == board[i + 1][col] == board[i + 2][col] == board[i + 3][col] != '-':
                return board[i][col]  # Return the winner ('X' or 'O')

    # Check diagonals (top-left to bottom-right)
    for i in range(3):
        for j in range(3):
            if board[i][j] == board[i + 1][j + 1] == board[i + 2][j + 2] == board[i + 3][j + 3] != '-':
                return board[i][j]  # Return the winner ('X' or 'O')

    # Check diagonals (top-right to bottom-left)
    for i in range(3):
        for j in range(3, 6):
            if board[i][j] == board[i + 1][j - 1] == board[i + 2][j - 2] == board[i + 3][j - 3] != '-':
                return board[i][j]  # Return the winner ('X' or 'O')

    return False  # No winner yet

def makeMove(board, move, player):
    playerPiece = ""
    if(player == 1):
        playerPiece = "X"
    else:
        playerPiece = "O"

    column = board[move]
    i = 0
    while(column[i] == "X" or column[i] == "O"):
        i += 1
        if(i>5):
            return False
   
    column[i] = playerPiece
    board[move] = column
    return board

def getValidMoves(board):
    valid_moves = [3, 2, 4, 1, 5, 0]  # Prefer the center column
    return [move for move in valid_moves if board[move][5] == "-"]


def evaluateBoard(board, player):
    def scoreSequence(sequence, player):
        opponent = "O" if player == "X" else "X"
        score = 0

        if sequence.count(player) == 4:
            score += 100
        elif sequence.count(player) == 3 and sequence.count("-") == 1:
            score += 10
        elif sequence.count(player) == 2 and sequence.count("-") == 2:
            score += 5

        if sequence.count(opponent) == 4:
            score -= 100
        elif sequence.count(opponent) == 3 and sequence.count("-") == 1:
            score -= 80

        return score

    score = 0

    # Score center column
    center_array = [board[3][i] for i in range(6)]
    center_count = center_array.count(player)
    score += center_count * 6

    # Score horizontal sequences
    for row in board:
        for col in range(6 - 3):
            sequence = row[col:col + 4]
            score += scoreSequence(sequence, player)

    # Score vertical sequences
    for col in range(6):
        for row in range(6 - 3):
            sequence = [board[row+i][col] for i in range(4)]
            score += scoreSequence(sequence, player)

    # Score diagonal sequences (top-left to bottom-right)
    for row in range(6 - 3):
        for col in range(6 - 3):
            sequence = [board[row+i][col+i] for i in range(4)]
            score += scoreSequence(sequence, player)

    # Score diagonal sequences (top-right to bottom-left)
    for row in range(6 - 3):
        for col in range(3, 6):
            sequence = [board[row+i][col-i] for i in range(4)]
            score += scoreSequence(sequence, player)

    return score


import time


def iterativeDeepening(board, maxDepth, maximizingPlayer, player, timeLimit):
    startTime = time.time()
    bestMove = None
    for depth in range(1, maxDepth + 1):
        elapsedTime = time.time() - startTime
        if elapsedTime > timeLimit:
            break
        score, move = minimax(board, depth, -math.inf, math.inf, maximizingPlayer, player)
        bestMove = move
    return bestMove


import math

def winningMove(board, piece):
    # Check horizontal locations for win
    for row in range(6):
        for col in range(3):
            if (board[col][row] == piece and board[col+1][row] == piece and
                board[col+2][row] == piece and board[col+3][row] == piece):
                return True

    # Check vertical locations for win
    for col in range(6):
        for row in range(3):
            if (board[col][row] == piece and board[col][row+1] == piece and
                board[col][row+2] == piece and board[col][row+3] == piece):
                return True

    # Check positively sloped diagonals
    for col in range(3):
        for row in range(3):
            if (board[col][row] == piece and board[col+1][row+1] == piece and
                board[col+2][row+2] == piece and board[col+3][row+3] == piece):
                return True

    # Check negatively sloped diagonals
    for col in range(3):
        for row in range(3, 6):
            if (board[col][row] == piece and board[col+1][row-1] == piece and
                board[col+2][row-2] == piece and board[col+3][row-3] == piece):
                return True

    return False



def findWinningMove(board, piece):
    for move in getValidMoves(board):
        newBoard = [col[:] for col in board]
        makeMove(newBoard, move, 1 if piece == "X" else 2)
        if winningMove(newBoard, piece):
            return move
    return None

def minimax(board, depth, alpha, beta, maximizingPlayer, player):
    opponent = "O" if player == "X" else "X"

    if depth == 0 or not getValidMoves(board):
        return evaluateBoard(board, player), None

    if maximizingPlayer:
        # Check for immediate winning move
        winning_move = findWinningMove(board, player)
        if winning_move is not None:
            return 1000, winning_move

        maxEval = -math.inf
        bestMove = None
        for move in getValidMoves(board):
            newBoard = [col[:] for col in board]
            makeMove(newBoard, move, 1 if player == "X" else 2)
            eval, _ = minimax(newBoard, depth - 1, alpha, beta, False, player)
            if eval > maxEval:
                maxEval = eval
                bestMove = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return maxEval, bestMove
    else:
        # Check for immediate blocking move
        blocking_move = findWinningMove(board, opponent)
        if blocking_move is not None:
            return -1000, blocking_move

        minEval = math.inf
        bestMove = None
        for move in getValidMoves(board):
            newBoard = [col[:] for col in board]
            makeMove(newBoard, move, 1 if player == "O" else 2)
            eval, _ = minimax(newBoard, depth - 1, alpha, beta, True, player)
            if eval < minEval:
                minEval = eval
                bestMove = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return minEval, bestMove

def playGame():
    initBoard = createBoard()
    printBoard(initBoard)
    board = initBoard
    turn = 0
    while(True):
        player = 1
        if turn % 2 == 0:
            player = 1
            while True:
                move = int(input("Player " + str(player) + " choose a column between 0-5\n"))
                if move < 0 or move > 5:
                    print("Please enter a value between 0-5")
                    continue
                newBoard = makeMove(board, move, player)
                if newBoard == False:
                    print("Cannot make that move")
                    continue
                else:
                    board = newBoard
                    printBoard(board)
                    break
        else:
            player = 2
            print("AI's turn...")
            _, move = minimax(board, 6, -math.inf, math.inf, True, "O")
            print(f"AI chose column {move}")
            board = makeMove(board, move, player)
            printBoard(board)
        
        turn += 1
        resultFromMove = gameHasEnded(board)
        if resultFromMove == False:
            continue
        else:
            print(f"{resultFromMove} wins!")
            break

playGame()





| - - - - - - |
| - - - - - - |
| - - - - - - |
| - - - - - - |
| - - - - - - |
| - - - - - - |


Player 1 choose a column between 0-5
 3


| - - - - - - |
| - - - - - - |
| - - - - - - |
| - - - - - - |
| - - - - - - |
| - - - X - - |
AI's turn...
AI chose column 2
| - - - - - - |
| - - - - - - |
| - - - - - - |
| - - - - - - |
| - - - - - - |
| - - O X - - |


Player 1 choose a column between 0-5
 3


| - - - - - - |
| - - - - - - |
| - - - - - - |
| - - - - - - |
| - - - X - - |
| - - O X - - |
AI's turn...
AI chose column 3
| - - - - - - |
| - - - - - - |
| - - - - - - |
| - - - O - - |
| - - - X - - |
| - - O X - - |


...

| - - - - - - |
| - - - - - - |
| - - - O - - |
| - X - O - - |
| - O X X - - |
| - X O X - - |
AI's turn...
AI chose column 3
| - - - - - - |
| - - - O - - |
| - - - O - - |
| - X - O - - |
| - O X X - - |
| - X O X - - |


Player 1 choose a column between 0-5
 3


| - - - X - - |
| - - - O - - |
| - - - O - - |
| - X - O - - |
| - O X X - - |
| - X O X - - |
AI's turn...
AI chose column 1
| - - - X - - |
| - - - O - - |
| - O - O - - |
| - X - O - - |
| - O X X - - |
| - X O X - - |


Player 1 choose a column between 0-5
 0


| - - - X - - |
| - - - O - - |
| - O - O - - |
| - X - O - - |
| - O X X - - |
| X X O X - - |
AI's turn...
AI chose column 0
| - - - X - - |
| - - - O - - |
| - O - O - - |
| - X - O - - |
| O O X X - - |
| X X O X - - |
